# Impacto del desbalanceo de datos 
lorem ipsum dolor sit amet, 

In [91]:
def reduction(malignant: int, decrement: float, i: int):
    return int(round(malignant - decrement * i))

In [2]:
benign = 2480
malignant = 5429
decrement = round((malignant - benign) * 0.05, 2)

print(f"Benign: {benign}, Malignant: {malignant}, Decrement: {decrement}")

output = malignant
i = 0
while output > benign:
    output = int(round(malignant - decrement * i))
    print(f"Benign: {benign}, Malignant: {output}")
    i += 1

Benign: 2480, Malignant: 5429, Decrement: 147.45
Benign: 2480, Malignant: 5429
Benign: 2480, Malignant: 5282
Benign: 2480, Malignant: 5134
Benign: 2480, Malignant: 4987
Benign: 2480, Malignant: 4839
Benign: 2480, Malignant: 4692
Benign: 2480, Malignant: 4544
Benign: 2480, Malignant: 4397
Benign: 2480, Malignant: 4249
Benign: 2480, Malignant: 4102
Benign: 2480, Malignant: 3954
Benign: 2480, Malignant: 3807
Benign: 2480, Malignant: 3660
Benign: 2480, Malignant: 3512
Benign: 2480, Malignant: 3365
Benign: 2480, Malignant: 3217
Benign: 2480, Malignant: 3070
Benign: 2480, Malignant: 2922
Benign: 2480, Malignant: 2775
Benign: 2480, Malignant: 2627
Benign: 2480, Malignant: 2480


In [93]:
# !bash move-images.sh

In [14]:
import glob
import numpy as np

from PIL import Image

# Cargamos el dataset como numpy array
def load_dataset(path: str) -> tuple[np.ndarray, np.ndarray]:
    images_path = glob.glob(f"{path}/*/*.png")

    images = []
    labels = []
    for image in images_path:
        label = 0 if image.split("/")[-2] == "benign" else 1
        img = Image.open(image)
        img = img.resize((64, 64))

        img = np.array(img) / 255.0

        images.append(img)
        labels.append(label)

    return np.array(images), np.array(labels)

X, y = load_dataset("./BreaKHis_v1/histology_slides/breast")

: 

: 

In [ ]:
print(X.shape, y.shape)

(7909, 64, 64, 3) (7909,)


In [ ]:
def create_balanceset(X: np.ndarray, y: np.ndarray, images: int):
    # Obtenemos los indices de las imágenes malignas
    malignant_indexes = np.where(y == 1)[0]

    # Elegimos i imagenes aleatorias
    malignant_indexes = np.random.choice(malignant_indexes, images, replace=False)

    # Obtenemos todas las imágenes benignas
    benign_indexes = np.where(y == 0)[0]

    # Concatenamos los indices
    indexes = np.concatenate((malignant_indexes, benign_indexes))

    # Obtenemos las imagenes y las etiquetas
    X = X[indexes]
    y = y[indexes]

    return X, y

# Ahora vamos a crear multiples datasets segun los porcentajes
def create_balanceset_2(X: np.ndarray, y: np.ndarray, percentage: int, malignant: int):
    # Obtenemos la candidad de clases
    benign = np.sum(y == 0)

    pctg = calculate_percentage(malignant, percentage)

    # Obtenemos la cantidad de imágenes que se tendran en el dataset
    malignant_images = malignant - pctg

    if malignant_images <= benign:
        malignant_images = 2480

    # Elegimos n (malignant_images) imagenes aleatorias de la clase maligna

    ## Obtenemos los indices de las imagenes malignas
    malignant_indexes = np.where(y == 1)[0]

    ## Elegimos n imagenes aleatorias
    malignant_indexes = np.random.choice(malignant_indexes, malignant_images, replace=False)

    ## Obtenemos todas las imágenes benignas
    benign_indexes = np.where(y == 0)[0]

    ## Concatenamos los indices
    indexes = np.concatenate((malignant_indexes, benign_indexes))

    ## Obtenemos las imagenes y las etiquetas
    X = X[indexes]
    y = y[indexes]

    return X, y, pctg

70 entrenamiento
20 prueba
10 validación

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
from collections import Counter


def create_torchset(X_np: np.ndarray, y_np: np.ndarray) -> TensorDataset:
    return TensorDataset(torch.from_numpy(X_np), torch.from_numpy(y_np))

def create_dataloader():
    pass

def split_dataset(data: TensorDataset) -> tuple:
    size = len(data)

    test_size = int(0.2 * size)
    train_val_size = size - test_size
    train_size = int(0.7 * train_val_size)
    val_size = train_val_size - train_size

    train_set, val_set, test_set = random_split(
        data, [train_size, val_size, test_size]
    )

    print(f"Número de muestras en el conjunto de entrenamiento: {len(train_set)}")
    print(f"Número de muestras en el conjunto de validación: {len(val_set)}")
    print(f"Número de muestras en el conjunto de prueba: {len(test_set)}")

    print(dict(Counter(train_set.targets)))

    return train_set, test_set, val_set

In [ ]:
benign = 2480
malignant = 5429
decrement = round((malignant - benign) * 0.05, 2)

output = malignant

i = 0
while output > benign:
    output = int(round(malignant - decrement * i))
    print(f"There are {benign} benign images and {output} malignant images")

    xx, yy = X, y

    X_reduced, y_reduced = create_balanceset(xx, yy, output)

    unique, counts = np.unique(y_reduced, return_counts=True)
    print(dict(zip(unique, counts)))

    dataset = create_torchset(X_reduced, y_reduced)

    train, test, val = split_dataset(dataset)

    i += 1

    break

There are 2480 benign images and 5429 malignant images


KeyboardInterrupt: 

In [ ]:


benign = np.sum(y == 0)
malignant = np.sum(y == 1)

print(f"Benign: {benign}, Malignant: {malignant}")
dataset = create_torchset(X, y)
train, test, val = split_dataset(dataset)

while malignant >= benign:
    xx, yy = X, y
    X_reduced, y_reduced, n = create_balanceset(xx, yy, 5, malignant)

    malignant -= n

    unique, counts = np.unique(y_reduced, return_counts=True)
    print(dict(zip(unique, counts)))

    # Ahora creamos un dataset con PyTorch
    dataset = create_torchset(X_reduced, y_reduced)

    train, test, val = split_dataset(dataset)
    


Benign: 2480, Malignant: 5429
Número de muestras en el conjunto de entrenamiento: 4429
Número de muestras en el conjunto de validación: 1899
Número de muestras en el conjunto de prueba: 1581
{0: 2480, 1: 5158}
Número de muestras en el conjunto de entrenamiento: 4277
Número de muestras en el conjunto de validación: 1834
Número de muestras en el conjunto de prueba: 1527
{0: 2480, 1: 4901}
Número de muestras en el conjunto de entrenamiento: 4133
Número de muestras en el conjunto de validación: 1772
Número de muestras en el conjunto de prueba: 1476
{0: 2480, 1: 4656}
Número de muestras en el conjunto de entrenamiento: 3996
Número de muestras en el conjunto de validación: 1713
Número de muestras en el conjunto de prueba: 1427
{0: 2480, 1: 4424}
Número de muestras en el conjunto de entrenamiento: 3866
Número de muestras en el conjunto de validación: 1658
Número de muestras en el conjunto de prueba: 1380
{0: 2480, 1: 4203}
Número de muestras en el conjunto de entrenamiento: 3742
Número de mue